# Import necessary packages

In [ ]:
from epx import Job, ModelConfig, SynthPop
import time

import pandas as pd
import numpy as np

import datetime as dt
import os
from pathlib import Path

import matplotlib.pyplot as plt
import matplotlib.patches as mpatches

import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
import requests

# Use the Epistemix default plotly template
r = requests.get("https://gist.githubusercontent.com/daniel-epistemix/8009ad31ebfa96ac97b7be038c014c0d/raw/320c3b0ca3dfbf7946e49c97254fa65d4753aeac/epx_plotly_theme.json")
if r.status_code == 200:
    pio.templates["epistemix"] = go.layout.Template(r.json())
    pio.templates.default = "epistemix"

import folium
from folium.plugins import TimestampedGeoJson
from folium.plugins import DualMap

os.environ["FRED_PROJECT"] = str(Path.cwd())
path_to_fred_project = os.environ["FRED_PROJECT"]

# Run FRED model

In [ ]:
finding_local_doctor_config = ModelConfig(
             synth_pop=SynthPop("US_2010.v5", ["Kewaunee_County_WI"]),
             start_date="2020-01-01",
             end_date="2020-02-01",
         )

finding_local_doctor_job = Job(
    "model/main.fred",
    config=[finding_local_doctor_config],
    key="cl_finding_local_doctor_job",
    fred_version="11.0.1",
    results_dir="/home/epx/cl-results"
)


finding_local_doctor_job.execute()

# the following loop idles while we wait for the simulation job to finish and periodically prints an update
update_count = 0
update_interval = 3
start_time = time.time()
timeout   = 300 # timeout in seconds
idle_time = 20   # time to wait (in seconds) before checking status again
while str(finding_local_doctor_job.status) != 'DONE':
    if str(finding_local_doctor_job.status) == 'ERROR':
        logs = finding_local_doctor_job.status.logs
        log_msg = "; ".join(logs.loc[logs.level == "ERROR"].message.tolist())
        print(f"Job failed with the following error:\n '{log_msg}'")
        break
    if time.time() > start_time + timeout:
        msg = f"Job did not finish within {timeout / 60} minutes."
        raise RuntimeError(msg)
    
    if update_count >= update_interval:
        update_count = 0
        print(f"Job is still processing after {time.time() - start_time:.0f} seconds")
        
    update_count += 1
    
    time.sleep(idle_time)

print(f"Job completed in {time.time() - start_time:.0f} seconds")

str(finding_local_doctor_job.status)

# Model introduction

The Dr Office modernisation efficiency model is a FRED demonstration model exploring the effect of modernisation of healthcare services on healthcare outcomes. At the individual level the model tracks personal health changes over time. Ideally, when patients become sick they try to access their local GP to resolve moderate health issues. For severe cases patients instead access the ED. 

A failure in the market can occur due to constraints in the system. An individual may ony visit their local GP and must 'queue' for an appointment on the day of their illness with all other individuals who need access to a Dr appointment that day. The number of avialable appointments on any given day is dictated by the 'modernity' of the office; whether the office can offer online, phone or only in-person apppointments. The more 'modern' the office, the greater the number of available appointments due to efficiency savings in online consulations. 

#### Individual actions:

Whilst a Dr Office may offer a range of appointments, individuals make their own choices based on their preferences. Person A may be happy to take any appointment they can get whilst Person B may only accept an in-person appointment. So what happens when Person B cannot access the in-person appointment they require? They get impatient. 

The threshold for impatience increases when a person's health is worse. For example, a person with health status 'mild' will attempt to book an appointment at their local Dr Office more days in a row before becoming impatient that someone with health status 'moderate'.

#### Population level outcomes:

When a person becomes impatient, they take accessing healthcare into their own hands and visit ED! The interplay between individual choices and Dr Office logistics lead to a global outcome of more (or less) incorrect visits to the ED affecting global waiting times for severe cases.

#### Some key model assumptions:

* Households only try to access appointments at their nearest Dr Office
* Households only try to access one appointment choice at the start of the day and do not rejoin the queue 
* The young and the elderly are more likely to be more severely ill


# Dr Office & Household locations

We can visualise the locations of Dr Offices in Kewaunee County, Wisconsin (colored rings) and the household locations of potential patients (white dots). 

In [ ]:
agent_dr_assignment = finding_local_doctor_job.results.csv_output("local_dr_office_assignment.csv")
agent_dr_assignment_wo_null_households = agent_dr_assignment[agent_dr_assignment.HH_LAT != -999]
# downsample household data for plotting
agent_dr_assignment_wo_null_households = agent_dr_assignment_wo_null_households[agent_dr_assignment_wo_null_households.HH_LON != -999][::10]

print("check; ", len(agent_dr_assignment_wo_null_households))
dr_office_data = pd.read_csv(f"{path_to_fred_project}/data/place_data_doctors.txt", sep=",")
dr_office_data.columns = ["ID", "LAT", "LON", "ELEV"]
dr_office_data.head()

# Define colour map for 
color_map = {
900000001:'#F0438D',
900000002:'#2BD6AF',
900000003:'#F9B72D'}


lat_cen = dr_office_data['LAT'].median()
long_cen = dr_office_data['LON'].median()

# Create interactive map with default basemap
map_osm = folium.Map(location=[lat_cen, long_cen],
                     tiles='https://api.mapbox.com/styles/v1/epxadmin/cm0ve9m13000501nq8q1zdf5p/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZXB4YWRtaW4iLCJhIjoiY20wcmV1azZ6MDhvcTJwcTY2YXpscWsxMSJ9._ROunfMS6hgVh1LPQZ4NGg',
                     attr='Mapbox',zoom_start = 12)

# place a ring at the location of the Dr Office and colour code using the color_map
for i, place_info in dr_office_data.iterrows():
    folium.Circle(radius=150,location=[place_info['LAT'], place_info['LON']], color=color_map[place_info['ID']]).add_to(map_osm)

# place a dot at the location of the agent households
for i, place_info in agent_dr_assignment_wo_null_households.iterrows():
    #if pd.notna(dr_office_data['LAT']):
    folium.Circle(radius=0.5,location=[place_info['HH_LAT'], place_info['HH_LON']], fill=True, color="#F9F6EE").add_to(map_osm)       
        
# display the map
map_osm

# Assigning Dr Office locations

Local Dr. assignments are made based on the absolute distance between the individual's household location and the Dr Office. We can visualise on our map which households were allocated to which Dr Office.

In [ ]:
dr_office_data = pd.read_csv(f"{path_to_fred_project}/data/place_data_doctors.txt", sep=",")
dr_office_data.columns = ["ID", "LAT", "LON", "ELEV"]
dr_office_data.head()

# Define colour map for 
color_map = {
900000001:'#F0438D',
900000002:'#2BD6AF',
900000003:'#F9B72D'}


lat_cen = dr_office_data['LAT'].median()
long_cen = dr_office_data['LON'].median()

# Create interactive map with default basemap
map_osm = folium.Map(location=[lat_cen, long_cen],
                     tiles='https://api.mapbox.com/styles/v1/epxadmin/cm0ve9m13000501nq8q1zdf5p/tiles/{z}/{x}/{y}?access_token=pk.eyJ1IjoiZXB4YWRtaW4iLCJhIjoiY20wcmV1azZ6MDhvcTJwcTY2YXpscWsxMSJ9._ROunfMS6hgVh1LPQZ4NGg',
                     attr='Mapbox',zoom_start = 11)

# place a ring at the location of the Dr Office and colour code using the color_map
for i, place_info in dr_office_data.iterrows():
    folium.Circle(radius=150,location=[place_info['LAT'], place_info['LON']], color=color_map[place_info['ID']]).add_to(map_osm)

# place a dot at the location of the agent households
for i, place_info in agent_dr_assignment_wo_null_households.iterrows():
    #if pd.notna(dr_office_data['LAT']):
    folium.Circle(radius=0.5,location=[place_info['HH_LAT'], place_info['HH_LON']], fill=True, color=color_map[place_info['OFFICE_ID']]).add_to(map_osm)       
        
# display the map
map_osm

# Individual access to healthcare

We can track individual health journeys over time and visualise these tracks. Below is a sample of 3 agent IDs - agent 227476979, agent 158002185 and agent 164118044. 

In [ ]:
agent_health = finding_local_doctor_job.results.csv_output("agent_health_tracks.csv")
agent_health.head()

labels=["healthy","mild","moderate","serious","severe"]
fig1, ax1 = plt.subplots(figsize=(10, 8))

selected_agent_ids = [227476979, 158002185, 164118044] 
cols = ['#25BEF4', '#96D656', '#A76FF4']       
                
for idx, agent_id in enumerate(selected_agent_ids):
    individual_health_df = agent_health.loc[agent_health['AGENT_ID'] == agent_id]
    x_date_trans = [dt.datetime.strptime(str(d),'%Y%m%d').date() for d in individual_health_df['DATE']]
    y = individual_health_df['HEALTH']
    x = x_date_trans
    ax1.plot(x, y, ls='--', c= cols[idx]) 
    ax1.scatter(x, y, label=agent_id, c= cols[idx]) 
plt.xticks(rotation = 45)    
ax1.set_yticks(np.arange(0,5))
ax1.set_yticklabels(labels)
plt.legend()
plt.show()

# Effect on global outcomes

We can visualise the number of patients visiting the ED over the course of the simulation. As a scenario planner, we would like to minimise the number of non-emergencies visiting the ED department so that resource for treating severe illness is optimised. What changes would you make to the model to track this?

In [ ]:
agent_at_ed = finding_local_doctor_job.results.csv_output("agent_at_ED.csv")
agent_at_ed.head()

labels=["healthy","mild","moderate","serious","severe"]

agent_count_health_at_ED = agent_at_ed.groupby(['HEALTH']).count().drop(columns=["DATE", "IMPATIENCE", "BOOKING_ATTEMPTS"]).reset_index("HEALTH")
agent_count_health_at_ED = agent_at_ed.drop(columns=["DATE", "IMPATIENCE", "BOOKING_ATTEMPTS"])

fig1, ax1 = plt.subplots(figsize=(10, 8))

emergency_patch = mpatches.Patch(color='#F0438D', label='Emergency')
non_emergency_patch = mpatches.Patch(color='#2BD6AF', label='Non-emergency')

bins=np.arange(-0.5, 4.5 + 1, 1)
counts, edges, bars = plt.hist(agent_count_health_at_ED[['HEALTH']],color='#2BD6AF', alpha=1, bins=bins, edgecolor='black')
ax1 = plt.gca()
# recolor the emergency bar
bars[-1].set_color('#F0438D')
bars[-1].set_edgecolor('black')

ax1.legend(handles=[emergency_patch, non_emergency_patch])
ax1.set_title("Agents arriving at ED for treatment",fontsize=14)
ax1.set_xticks(np.arange(0,5))
ax1.set_xticklabels(labels)
ax1.set_xlabel("Health")
ax1.set_ylabel("Count")
plt.show()



In [ ]:
finding_local_doctor_job.delete(interactive=False)